## 02 Data Processing

#### Import relevant libraries

In [2]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

#### Load raw data

In [3]:
input_dir = os.path.join('..', 'data', 'original')
exp_df = pd.read_csv(os.path.join(input_dir, 'expenditure_bills_burden.csv'))
hui_df = pd.read_csv(os.path.join(input_dir, 'housing_units_income.csv'))
aei_df = pd.read_csv(os.path.join(input_dir, 'assets_earnings_investments.csv'))
sales_df = pd.read_csv(os.path.join(input_dir, 'customers_sales.csv'))

In [4]:
exp_df.head()

,parent_name,utility_name,respondent_id,year,percent_AMI,ownership,electricity_gas_other,technology,expenditure,bill,burden
0,Southern Co.,Alabama Power Co.,2.0,2020,0-30%,owner,Electricity,adjustment,9.276270e+06,8.394506,0.009773
1,Southern Co.,Alabama Power Co.,2.0,2020,0-30%,owner,Electricity,distribution,3.017570e+07,27.307328,0.031792
2,Southern Co.,Alabama Power Co.,2.0,2020,0-30%,owner,Electricity,hydro,4.312818e+06,3.902860,0.004544
3,Southern Co.,Alabama Power Co.,2.0,2020,0-30%,owner,Electricity,nuclear,1.912960e+07,17.311221,0.020154
4,Southern Co.,Alabama Power Co.,2.0,2020,0-30%,owner,Electricity,other,1.578084e+07,14.280782,0.016626


In [5]:
hui_df.head()

,parent_name,utility_name,respondent_id,year,percent_AMI,ownership,housing_units,income
0,Southern Co.,Alabama Power Co.,2.0,2020,0-30%,owner,92086.710664,9.491514e+08
1,Southern Co.,Alabama Power Co.,2.0,2020,0-30%,renter,126159.321739,1.238083e+09
2,Southern Co.,Alabama Power Co.,2.0,2020,100%+,owner,521425.299562,6.049073e+10
3,Southern Co.,Alabama Power Co.,2.0,2020,100%+,renter,111421.757248,9.494001e+09
4,Southern Co.,Alabama Power Co.,2.0,2020,30-60%,owner,108469.889403,2.541532e+09


In [6]:
aei_df.head()

,parent_name,utility_name,respondent_id,year,asset,sub_asset,asset_value,earnings_value,investment_value
0,"American Electric Power Co., Inc.",AEP Generating Co.,1.0,2020,other,AROs,-15405378.0,-0.0,NaN
1,"American Electric Power Co., Inc.",AEP Generating Co.,1.0,2020,other,electric_plant_held_for_future_use,1034099.0,0.0,NaN
2,"American Electric Power Co., Inc.",AEP Generating Co.,1.0,2020,other,electric_plant_leased_to_others,155616036.0,0.0,NaN
3,"American Electric Power Co., Inc.",AEP Generating Co.,1.0,2020,other,general_plant,427318.0,0.0,1846.0
4,"American Electric Power Co., Inc.",AEP Generating Co.,1.0,2020,other,intangible_plant,724784.0,0.0,NaN


In [7]:
sales_df.head()

,parent_name,utility_name,respondent_id,year,customer_type,customer_type_rmi,customers,sales,revenues
0,"American Electric Power Co., Inc.",AEP Generating Co.,1,2020,sales_for_resale,sales_for_resale,NaN,2019392.0,246848141.0
1,"American Electric Power Co., Inc.",AEP Generating Co.,1,2019,sales_for_resale,sales_for_resale,NaN,4073472.0,306947627.0
2,"American Electric Power Co., Inc.",AEP Generating Co.,1,2018,sales_for_resale,sales_for_resale,2.0,5947055.0,339868815.0
3,"American Electric Power Co., Inc.",AEP Generating Co.,1,2017,sales_for_resale,sales_for_resale,2.0,6069003.0,340871490.0
4,"American Electric Power Co., Inc.",AEP Generating Co.,1,2016,sales_for_resale,sales_for_resale,3.0,13491086.0,563840279.0


#### Formatting data

(a) Rename columns

In [8]:
exp_df = exp_df.rename({'percent_AMI': 'percent_ami'}, axis='columns')
hui_df = hui_df.rename({'percent_AMI': 'percent_ami'}, axis='columns')

(b) Data type formatting

In [9]:
exp_df.dtypes

parent_name               object
utility_name              object
respondent_id            float64
year                       int64
percent_ami               object
ownership                 object
electricity_gas_other     object
technology                object
expenditure              float64
bill                     float64
burden                   float64
dtype: object

In [10]:
hui_df.dtypes

parent_name       object
utility_name      object
respondent_id    float64
year               int64
percent_ami       object
ownership         object
housing_units    float64
income           float64
dtype: object

In [11]:
aei_df.dtypes

parent_name          object
utility_name         object
respondent_id       float64
year                  int64
asset                object
sub_asset            object
asset_value         float64
earnings_value      float64
investment_value    float64
dtype: object

In [12]:
sales_df.dtypes

parent_name           object
utility_name          object
respondent_id          int64
year                   int64
customer_type         object
customer_type_rmi     object
customers            float64
sales                float64
revenues             float64
dtype: object

#### Join datasets

Prepare expenditure_bills_burden.csv data frame

In [13]:
# make a copy
exp_prepared_df = exp_df.copy()

In [14]:
# groupby utility x year
exp_prepared_df = exp_prepared_df.groupby(['respondent_id', 'year', 'technology'])\
    .agg({'bill': ['sum'], 'expenditure': ['sum']})\
    .reset_index()
exp_prepared_df = exp_prepared_df.droplevel(1, axis=1)

In [15]:
# pivot by technology 
exp_prepared_df = exp_prepared_df.pivot(
        index=['respondent_id', 'year'],
        columns='technology',
        values=['bill', 'expenditure']
)
exp_prepared_df.columns = ['_'.join(col).strip() for col in exp_prepared_df.columns.values]
exp_prepared_df.columns = exp_prepared_df.columns.str.lower()
exp_prepared_df.columns = exp_prepared_df.columns.to_flat_index()
exp_prepared_df = exp_prepared_df.reset_index()

In [16]:
exp_prepared_df.head()

,respondent_id,year,bill_gas,bill_other fuel,bill_adjustment,bill_distribution,bill_hydro,bill_nuclear,bill_other,bill_other_fossil,...,expenditure_adjustment,expenditure_distribution,expenditure_hydro,expenditure_nuclear,expenditure_other,expenditure_other_fossil,expenditure_purchased_power,expenditure_renewables,expenditure_steam,expenditure_transmission
0,2.0,2005,375.031464,7.084629,9.540319,140.598363,24.935923,79.832394,93.020037,115.705892,...,1.425108e+07,2.100222e+08,3.724864e+07,1.192516e+08,1.389510e+08,1.728385e+08,1.870094e+08,0.0,4.974018e+08,9.923683e+07
1,2.0,2006,403.967434,8.175744,20.977633,150.312633,25.845165,83.786720,112.218061,136.302804,...,3.165222e+07,2.268000e+08,3.899662e+07,1.264220e+08,1.693208e+08,2.056612e+08,1.829700e+08,0.0,5.747494e+08,1.077316e+08
2,2.0,2007,362.674741,9.373432,25.958163,162.860281,26.922500,86.942468,136.816315,146.322099,...,3.959165e+07,2.483961e+08,4.106246e+07,1.326055e+08,2.086736e+08,2.231719e+08,2.064529e+08,0.0,6.174916e+08,1.161173e+08
3,2.0,2008,389.722554,12.139279,17.184093,161.259248,25.216835,87.957893,136.354090,157.862545,...,2.642192e+07,2.479490e+08,3.877291e+07,1.352423e+08,2.096554e+08,2.427263e+08,2.337353e+08,0.0,7.437904e+08,1.193094e+08
4,2.0,2009,368.234034,10.900991,22.232564,178.909959,25.595063,94.547352,144.596841,136.960261,...,3.462866e+07,2.786639e+08,3.986597e+07,1.472636e+08,2.252190e+08,2.133245e+08,1.387760e+08,0.0,7.525156e+08,1.314208e+08


Prepare housing_units_income.csv data frame

In [17]:
# make a copy
hui_prepared_df = hui_df.copy()

In [18]:
# drop repeated columns
hui_prepared_df = hui_prepared_df.drop(columns=[
    'parent_name', 
    'utility_name', 
    'ownership',
])

In [19]:
# groupby utility x year
hui_prepared_df = hui_prepared_df.groupby(['respondent_id', 'year'])\
    .agg({'housing_units': ['sum'], 'income': ['sum']})\
    .reset_index()
hui_prepared_df = hui_prepared_df.droplevel(1, axis=1)

In [20]:
hui_prepared_df.head()

,respondent_id,year,housing_units,income
0,2.0,2005,1177707.0,6.002705e+10
1,2.0,2006,1189597.0,6.287944e+10
2,2.0,2007,1202491.0,6.652127e+10
3,2.0,2008,1212244.0,6.973237e+10
4,2.0,2009,1228000.0,6.732521e+10


Prepare assets_earnings_income.csv data frame

In [21]:
# make a copy
aei_prepared_df = aei_df.copy()

In [22]:
# drop repeated columns
aei_prepared_df = aei_prepared_df.drop(columns=[
    'parent_name', 
    'utility_name', 
])

In [23]:
# groupby utility x year
aei_prepared_df = aei_prepared_df.groupby(['respondent_id', 'year', 'asset'])\
    .agg({'asset_value': ['sum'], 'earnings_value': ['sum'], 'investment_value': ['sum']})\
    .reset_index()
aei_prepared_df = aei_prepared_df.droplevel(1, axis=1)

In [24]:
# pivot by asset 
aei_prepared_df = aei_prepared_df.pivot(
        index=['respondent_id', 'year'],
        columns='asset',
        values=['asset_value', 'earnings_value', 'investment_value']
)
aei_prepared_df.columns = ['_'.join(col).strip() for col in aei_prepared_df.columns.values]
aei_prepared_df.columns = aei_prepared_df.columns.str.lower()
aei_prepared_df.columns = aei_prepared_df.columns.to_flat_index()
aei_prepared_df = aei_prepared_df.reset_index()

In [25]:
aei_prepared_df.head()

,respondent_id,year,asset_value_distribution,asset_value_hydro,asset_value_nuclear,asset_value_other,asset_value_other_fossil,asset_value_renewables,asset_value_steam,asset_value_transmission,...,earnings_value_steam,earnings_value_transmission,investment_value_distribution,investment_value_hydro,investment_value_nuclear,investment_value_other,investment_value_other_fossil,investment_value_renewables,investment_value_steam,investment_value_transmission
0,1.0,2005,NaN,NaN,NaN,-1.640519e+08,NaN,NaN,2.656242e+08,NaN,...,0.0,NaN,NaN,NaN,NaN,321685.0,NaN,NaN,10648224.0,NaN
1,1.0,2006,NaN,NaN,NaN,-1.388877e+08,NaN,NaN,2.506720e+08,NaN,...,0.0,NaN,NaN,NaN,NaN,321958.0,NaN,NaN,7159107.0,NaN
2,1.0,2007,NaN,NaN,NaN,-7.481310e+07,307116436.0,NaN,2.684942e+08,NaN,...,0.0,NaN,NaN,NaN,NaN,2958905.0,0.0,NaN,36888943.0,NaN
3,1.0,2008,NaN,NaN,NaN,7.355553e+07,308800985.0,NaN,2.597126e+08,NaN,...,0.0,NaN,NaN,NaN,NaN,3109017.0,0.0,NaN,15305385.0,NaN
4,1.0,2009,NaN,NaN,NaN,7.225845e+07,303881470.0,NaN,2.656392e+08,5153527.0,...,0.0,0.0,NaN,NaN,NaN,1553733.0,0.0,NaN,34984605.0,1937818.0


Prepare customer_sales.csv data frame

In [26]:
# make a copy
sales_prepared_df = sales_df.copy()

In [27]:
# drop repeated columns
sales_prepared_df = sales_prepared_df.drop(columns=[
    'parent_name',
    'utility_name',
])

In [28]:
# groupby utility x year
sales_prepared_df = sales_prepared_df.groupby(['respondent_id', 'year'])\
    .agg({'customers': ['sum'], 'revenues': ['sum'], 'sales': ['sum']})\
    .reset_index()
sales_prepared_df = sales_prepared_df.droplevel(1, axis=1)

In [29]:
sales_prepared_df.head()

,respondent_id,year,customers,revenues,sales
0,1,2005,0.0,270544818.0,8969040.0
1,1,2006,2.0,309603782.0,10276134.0
2,1,2007,3.0,380328754.0,9027362.0
3,1,2008,3.0,468063360.0,10622505.0
4,1,2009,3.0,414572117.0,9914827.0


Join datasets into one dataframe

In [30]:
raw_df = pd.merge(exp_prepared_df, hui_prepared_df, how='inner', on=['respondent_id', 'year'])
raw_df = pd.merge(raw_df, aei_prepared_df, how='inner', on=['respondent_id', 'year'])
raw_df = pd.merge(raw_df, sales_prepared_df, how='inner', on=['respondent_id', 'year'])
raw_df.head()

,respondent_id,year,bill_gas,bill_other fuel,bill_adjustment,bill_distribution,bill_hydro,bill_nuclear,bill_other,bill_other_fossil,...,investment_value_hydro,investment_value_nuclear,investment_value_other,investment_value_other_fossil,investment_value_renewables,investment_value_steam,investment_value_transmission,customers,revenues,sales
0,2.0,2005,375.031464,7.084629,9.540319,140.598363,24.935923,79.832394,93.020037,115.705892,...,6223361.0,46613178.0,53947740.0,0.0,NaN,323629554.0,116211224.0,1394751.0,4.461784e+09,76622962.0
1,2.0,2006,403.967434,8.175744,20.977633,150.312633,25.845165,83.786720,112.218061,136.302804,...,9929295.0,16458291.0,45795288.0,0.0,NaN,313129533.0,110999847.0,1409748.0,4.846311e+09,77363425.0
2,2.0,2007,362.674741,9.373432,25.958163,162.860281,26.922500,86.942468,136.816315,146.322099,...,3998749.0,33368124.0,84124085.0,0.0,NaN,221428399.0,141334126.0,1425243.0,5.176492e+09,75564806.0
3,2.0,2008,389.722554,12.139279,17.184093,161.259248,25.216835,87.957893,136.354090,157.862545,...,7887843.0,25021763.0,-17035082.0,0.0,NaN,661587762.0,134557031.0,1435370.0,5.882591e+09,75436226.0
4,2.0,2009,368.234034,10.900991,22.232564,178.909959,25.595063,94.547352,144.596841,136.960261,...,5179286.0,36619331.0,70684448.0,0.0,NaN,479854652.0,159051998.0,1435611.0,5.354824e+09,71369366.0


#### Data cleaning

In [31]:
# make a copy
clean_df = raw_df.copy()

(a) Impute values for missing data

In [32]:
clean_df = clean_df.fillna(0) 

(b) Remove probable outliers

In [33]:
# negative investments ...
# align naming for assets = technology ...

(c) Compute consumer energy burden

In [34]:
clean_df['energy_burden'] = clean_df['revenues'] / clean_df['income']

(d) Compute normalized expenditure (price)

In [35]:
clean_df['normalized_expenditure'] = clean_df['revenues'] / clean_df['sales'] 

#### Save processed_data file

In [36]:
output_dir = os.path.join('..', 'data', 'processed')

In [37]:
clean_df.to_csv(os.path.join(output_dir, 'processed_data.csv'))